In [3]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
585,Quentin Tarantino's partner in crime Roger Ava...,negative
428,"I have a friend that works at blockbuster, and...",negative
836,"I know it's not original, but what the hey? Wh...",negative
597,"Yes, it might be not historically accurate(act...",negative
308,"Definitely a very good idea,screenplay was jus...",positive


In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_232120/3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [8]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/sei/nltk_data...


True

In [9]:
df = normalize_text(df)
df.head()

,review,sentiment
585,quentin tarantino partner crime roger avary co...,negative
428,friend work blockbuster get free movie rental ...,negative
836,know original hey else said feel unutterably s...,negative
597,yes might historically accurate actually soldi...,negative
308,definitely good idea screenplay ok could bette...,positive


In [10]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [11]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [12]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
585,quentin tarantino partner crime roger avary co...,0
428,friend work blockbuster get free movie rental ...,0
836,know original hey else said feel unutterably s...,0
597,yes might historically accurate actually soldi...,0
308,definitely good idea screenplay ok could bette...,1


In [13]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/seika-afk/MLOPS-project2-.mlflow')
dagshub.init(repo_owner='seika-afk', repo_name='MLOPS-project2-', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d49bd491-b1ea-4ef9-a460-a095b2d2d33d&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=1d4465d94d94d123841d584850145e6aa7c5b7f0367f3168c0c6652ca34c8586




Output()

Opening in existing browser session.


Accessing as seika-afk

Initialized MLflow to track repo "seika-afk/MLOPS-project2-"

Repository seika-afk/MLOPS-project2- initialized!

2025/08/15 15:17:49 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/4bb4ba45bd874834af187d4b2e74f9ae', creation_time=1755251269990, experiment_id='0', last_update_time=1755251269990, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [18]:
import mlflow
import mlflow.sklearn
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        # Log preprocessing parameters
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        # Initialize and train model
        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)
        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        # Log model parameters
        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        # Make predictions
        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        # Calculate evaluation metrics
        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # Log evaluation metrics
        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # Save and log the model locally as an artifact
        logging.info("Saving and logging the model locally...")
        local_model_path = "local_model"
        mlflow.sklearn.save_model(model, local_model_path)
        mlflow.log_artifact(local_model_path)

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Print results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-08-15 15:20:58,136 - INFO - Starting MLflow run...
2025-08-15 15:20:58,818 - INFO - Logging preprocessing parameters...
2025-08-15 15:21:00,352 - INFO - Initializing Logistic Regression model...
2025-08-15 15:21:00,353 - INFO - Fitting the model...
2025-08-15 15:21:00,628 - INFO - Model training complete.
2025-08-15 15:21:00,628 - INFO - Logging model parameters...
2025-08-15 15:21:01,069 - INFO - Making predictions...
2025-08-15 15:21:01,072 - INFO - Calculating evaluation metrics...
2025-08-15 15:21:01,080 - INFO - Logging evaluation metrics...
2025-08-15 15:21:07,418 - INFO - Saving and logging the model locally...
2025-08-15 15:21:51,046 - INFO - Model training and logging completed in 52.23 seconds.
2025-08-15 15:21:51,047 - INFO - Accuracy: 0.584
2025-08-15 15:21:51,047 - INFO - Precision: 0.5970149253731343
2025-08-15 15:21:51,048 - INFO - Recall: 0.6153846153846154
2025-08-15 15:21:51,048 - INFO - F1 Score: 0.6060606060606061


🏃 View run big-ray-346 at: https://dagshub.com/seika-afk/MLOPS-project2-.mlflow/#/experiments/0/runs/ef715f16d6d8458681296e8fdb31c404
🧪 View experiment at: https://dagshub.com/seika-afk/MLOPS-project2-.mlflow/#/experiments/0
